In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


/Users/mustainbillah/Projects/sentiment-analysis/venv-sentiment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the tracking URI to the local mlruns directory
mlflow.set_tracking_uri("file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns")

# Create and set the experiment
experiment_name = "04_ML_Algorithm_Comparison_with_HPT"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns/658432777505770578', creation_time=1748791728665, experiment_id='658432777505770578', last_update_time=1748791728665, lifecycle_stage='active', name='04_ML_Algorithm_Comparison_with_HPT', tags={}>

In [3]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [4]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for KNN

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)  # Tuning the number of neighbors
    p = trial.suggest_categorical('p', [1, 2])  # Tuning the distance metric (1 for Manhattan, 2 for Euclidean)

    # KNeighborsClassifier setup
    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for KNN
run_optuna_experiment()


[I 2025-06-02 08:27:33,967] A new study created in memory with name: no-name-933a6d64-9d72-4395-bd40-6b2f5675b529
[I 2025-06-02 08:27:39,543] Trial 0 finished with value: 0.5527372648488692 and parameters: {'n_neighbors': 16, 'p': 2}. Best is trial 0 with value: 0.5527372648488692.
[I 2025-06-02 08:27:44,988] Trial 1 finished with value: 0.5826463749735785 and parameters: {'n_neighbors': 4, 'p': 2}. Best is trial 1 with value: 0.5826463749735785.
[I 2025-06-02 08:27:47,439] Trial 2 finished with value: 0.3925174381737476 and parameters: {'n_neighbors': 8, 'p': 1}. Best is trial 1 with value: 0.5826463749735785.
[I 2025-06-02 08:27:49,852] Trial 3 finished with value: 0.380997674910167 and parameters: {'n_neighbors': 19, 'p': 1}. Best is trial 1 with value: 0.5826463749735785.
[I 2025-06-02 08:27:56,706] Trial 4 finished with value: 0.5262101035721835 and parameters: {'n_neighbors': 28, 'p': 2}. Best is trial 1 with value: 0.5826463749735785.
[I 2025-06-02 08:28:02,602] Trial 5 finished